In [199]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas
import numpy as np
import requests
import warnings
warnings.filterwarnings('ignore');

import osmnx as ox
import folium

import os
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## p_acquisition

In [49]:
#importar dataset parques municipales JSON
json_parques =  requests.get("https://datos.madrid.es/egob/catalogo/200761-0-parques-jardines.json").json()["@graph"]
json_parques_df = pd.json_normalize(json_parques)
json_parques_df
#solo para ver si funciona, voy a utilizar csv

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,6070995,Jardines Gregorio Ordóñez,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28006,CALLE PRINCIPE DE VERGARA 68,40.433462,-3.678595,Jardines en memoria de Gregorio Ordóñez. Este ...,2,,,Jardines Gregorio Ordóñez
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,6010361,Jardines San Francisco el Grande,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,AVENIDA GRAN VIA DE SAN FRANCISCO 29,40.407143,-3.712167,Este jardín se encuentra en el solar de 4.200 ...,2,,,Jardines San Francisco el Grande
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5964519,Jardines de Andrés Saborit,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28003,CALLE RAIMUNDO FERNANDEZ VILLAVERDE 32,40.446133,-3.698970,Jardines situados en la calle Raimundo Fernánd...,2,,Zonas infantiles Zonas de mayores Circuito Bio...,Jardines de Andrés Saborit
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5896013,Jardines de El Buen Retiro,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28001,PLAZA INDEPENDENCIA 7,40.419641,-3.687934,Jardín Histórico Artístico creado en el siglo ...,2,Horario del parque Primavera y verano (de a...,Zonas infantiles: Zona del Pinar (Puerta del ...,Jardines de El Buen Retiro
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,NaN,10984428,Jardines de Gloria Fuertes,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28016,CALLE MADRE DE DIOS 2,40.461518,-3.675118,Jardín dedicado a la poetisa y escritora de cu...,2,,Zona infantil,Jardines de Gloria Fuertes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,6028228,Vivero Estufas de El Retiro,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28009,GLORIETA ANGEL CAIDO 4 Parque del Retiro. Entr...,40.409899,-3.682331,Recinto construido en 1889 para albergar diver...,0,Invierno de 7 a 23 horas. El acceso al público...,,Vivero Estufas de El Retiro
199,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,6028009,Vivero de la Casa de Campo,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28011,PASEO AZUL 2,40.419842,-3.728697,La primera noticia documental la tenemos en 18...,2,Invierno de 8 a 15 horas Verano de 7 a 14 hora...,,Vivero de la Casa de Campo
200,https://datos.madrid.es/egob/catalogo/tipo/ent...,NaN,6028007,Vivero municipal de Migas Calientes,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28040,AUTOVIA M-30 2200,40.445356,-3.739974,Centro de producción donde se realiza el culti...,0,De 8 a 15 horas.,,Vivero municipa

In [50]:
#importar dataset parques municipales CSV
parques_municipales_df = pd.read_csv("datasets/parques_municipales.csv", sep=';')
parques_municipales_df

,PK,NOMBRE,DESCRIPCION-ENTIDAD,HORARIO,EQUIPAMIENTO,TRANSPORTE,DESCRIPCION,ACCESIBILIDAD,CONTENT-URL,NOMBRE-VIA,...,DISTRITO,COORDENADA-X,COORDENADA-Y,LATITUD,LONGITUD,TELEFONO,FAX,EMAIL,TIPO,Unnamed: 30
0,6070995,Jardines Gregorio Ordóñez,Jardines en memoria de Gregorio Ordóñez. Este ...,NaN,NaN,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",Conservación: Dirección General de Gestión del...,2,http://www.madrid.es/sites/v/index.jsp?vgnextc...,PRINCIPE DE VERGARA,...,SALAMANCA,442443.0,4476090.0,40.433462,-3.678595,NaN,NaN,NaN,/contenido/entidadesYorganismos/ParquesJardines,
1,6010361,Jardines San Francisco el Grande,Este jardín se encuentra en el solar de 4.200 ...,NaN,NaN,"Bus: 3, 60 y 148. Servicio Bicimad, para cons...",Conservación:Dirección General de Gestión del ...,2,http://www.madrid.es/sites/v/index.jsp?vgnextc...,GRAN VIA DE SAN FRANCISCO,...,CENTRO,439572.0,4473192.0,40.407143,-3.712167,NaN,NaN,NaN,/contenido/entidadesYorganismos/ParquesJardine...,
2,5964519,Jardines de Andrés Saborit,Jardines situados en la calle Raimundo Fernánd...,NaN,Zonas infantiles Zonas de mayores Circuito Bio...,"Metro: Nuevos Ministerios (líneas 6, 8 y 10) B...",Conservación: Dirección General de Gestión del...,2,http://www.madrid.es/sites/v/index.jsp?vgnextc...,RAIMUNDO FERNANDEZ VILLAVERDE,...,CHAMBERI,440726.0,4477510.0,40.446133,-3.698970,NaN,NaN,NaN,/contenido/entidadesYorganismos/ParquesJardines,
3,5896013,Jardines de El Buen Retiro,Jardín Histórico Artístico creado en el siglo ...,Horario del parque Primavera y verano (de a...,Zonas infantiles: Zona del Pinar (Puerta del ...,"Metro: Retiro (línea 2), Príncipe de Vergara (...",Conservación: Dirección General de Gestión del...,2,http://www.madrid.es/sites/v/index.jsp?vgnextc...,INDEPENDENCIA,...,RETIRO,441640.0,4474563.0,40.419641,-3.687934,915 744 024 (taquillas embarcadero) / 915 280 ...,NaN,NaN,/contenido/entidadesYorganismos/ParquesJardine...,
4,10984428,Jardines de Gloria Fuertes,Jardín dedicado a la poetisa y escritora de cu...,NaN,Zona infantil,"Metro: línea 9 (Colombia) Bus: 16,29,150 Serv...",Conservación: Dirección General de Gestión del...,2,http://www.madrid.es/sites/v/index.jsp?vgnextc...,MADRE DE DIOS,...,CHAMARTIN,442761.0,4479202.0,40.461518,-3.675118,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,6028228,Vivero Estufas de El Retiro,Recinto construido en 1889 para albergar diver...,Invierno de 7 a 23 horas. El acceso al público...,NaN,"Metro: Atocha (línea 1), Ibiza (línea 9) Bus: ...",Conservación: Dirección General de Gestión del...,0,http://www.madrid.es/sites/v/index.jsp?vgnextc...,ANGEL CAIDO,...,RETIRO,442107.0,4473478.0,40.409899,-3.682331,91 5 28 09 38,NaN,viveroestufasretiro@madrid.es,/contenido/entidadesYorganismos/ParquesJardines,
199,6028009,Vivero de la Casa de Campo,La primera noticia documental la tenemos en 18...,Invierno de 8 a 15 horas Verano de 7 a 14 hora...,NaN,"Metro: Príncipe Pío (líneas 6 y 10 y ramal), L...",Conservación: Dirección General de Gestión del...,2,http://www.madrid.es/sites/v/index.jsp?vgnextc...,AZUL,...,MONCLOA-ARAVACA,438182.0,4474613.0,40.419842,-3.728697,91.526.77.03 o 91.480.19.57,NaN,viverocasacampo@madrid.es o infocasacampo@mad...,/contenido/entidadesYorganismos/ParquesJardines,
200,6028007,Vivero municipal de Migas Calientes,Centro de producción donde se realiza el culti...,De 8 a 15 horas.,NaN,NaN,Conservación: Dirección General de Gestión del...,0,http://www.madrid.es/sites/v/index.jsp?vgnextc...,M-30,...,MONCLOA-ARAVACA,437249.0,4477453.0,40.445356,-3.739974,915490676,NaN,viveromigascalientes@madrid.es,NaN,
201,6126515,Zona verde C/ Fresnedillas,Esta nueva zona verde está situada entre las c...,NaN,Zona de juegos infantiles Área de juegos para ...,NaN,Conservación:Dirección General de Gestión del ...,2,http://www.madrid.es/sites/v/index.jsp?vgnextc...,FRESNEDILLAS,...,FUENCARRAL-EL PARDO,437021.0,4480996.0,40.477264,-3.743003,NaN,NaN,NaN,/contenido/entidad

In [51]:
#parques_municipales_df.iloc[0,:]

In [52]:
#importar bicimad
bicimad_st_df = pd.read_csv("datasets/bicimad_statios_database.csv", sep=',')
bicimad_st_df

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry_type,geometry_coordinates
0,1,Puerta del Sol A,2,1a,Puerta del Sol nº 1,1,0,30,15,11,0,Point,"[-3.7018341, 40.4172137]"
1,2,Puerta del Sol B,1,1b,Puerta del Sol nº 1,1,0,30,22,4,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,Miguel Moya,2,2,Calle Miguel Moya nº 1,1,0,24,12,11,0,Point,"[-3.7058415, 40.4205886]"
3,4,Plaza Conde Suchil,0,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,4,11,0,Point,"[-3.7069171, 40.4302937]"
4,5,Malasaña,2,4,Calle Manuela Malasaña nº 5,1,0,24,12,12,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,266,Ciudad Universitaria 1,0,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,0,24,0,Point,"[-3.72699, 40.44375]"
260,267,Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,12,0,Point,"[-3.72693, 40.44342]"
261,268,Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,9,14,0,Point,"[-3.7272945, 40.4483322]"
262,269,Facultad Derecho,0,261,Avenida Complutense nº 23,1,0,24,5,18,0,Point,"[-3.72937, 40.45109]"


## p_analysis

#### completar df parques municipales

In [53]:
parques_municipales_df["Type of Place"] = "Principales parques y jardines municipales"
#parques_municipales_df

#### obtener latitud y longitud de bicimad

In [54]:
def bicimad_geo_long(geo):
    geo_clean = geo.replace("[","").replace("]","").split(",")
    longitud = geo_clean[0].strip()
    return longitud

In [55]:
bicimad_st_df["LONGITUD"] = bicimad_st_df["geometry_coordinates"].apply(bicimad_geo_long)
bicimad_st_df.head()

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry_type,geometry_coordinates,LONGITUD
0,1,Puerta del Sol A,2,1a,Puerta del Sol nº 1,1,0,30,15,11,0,Point,"[-3.7018341, 40.4172137]",-3.7018341
1,2,Puerta del Sol B,1,1b,Puerta del Sol nº 1,1,0,30,22,4,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701602938060457
2,3,Miguel Moya,2,2,Calle Miguel Moya nº 1,1,0,24,12,11,0,Point,"[-3.7058415, 40.4205886]",-3.7058415
3,4,Plaza Conde Suchil,0,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,4,11,0,Point,"[-3.7069171, 40.4302937]",-3.7069171
4,5,Malasaña,2,4,Calle Manuela Malasaña nº 5,1,0,24,12,12,0,Point,"[-3.7025875, 40.4285524]",-3.7025875


In [56]:
def bicimad_geo_lat(geo):
    geo_clean = geo.replace("[","").replace("]","").split(",")
    latitud = geo_clean[1].strip()
    return latitud

In [57]:
bicimad_st_df["LATITUD"] = bicimad_st_df["geometry_coordinates"].apply(bicimad_geo_lat)
bicimad_st_df.head()

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry_type,geometry_coordinates,LONGITUD,LATITUD
0,1,Puerta del Sol A,2,1a,Puerta del Sol nº 1,1,0,30,15,11,0,Point,"[-3.7018341, 40.4172137]",-3.7018341,40.4172137
1,2,Puerta del Sol B,1,1b,Puerta del Sol nº 1,1,0,30,22,4,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701602938060457,40.41731271011562
2,3,Miguel Moya,2,2,Calle Miguel Moya nº 1,1,0,24,12,11,0,Point,"[-3.7058415, 40.4205886]",-3.7058415,40.4205886
3,4,Plaza Conde Suchil,0,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,4,11,0,Point,"[-3.7069171, 40.4302937]",-3.7069171,40.4302937
4,5,Malasaña,2,4,Calle Manuela Malasaña nº 5,1,0,24,12,12,0,Point,"[-3.7025875, 40.4285524]",-3.7025875,40.4285524


#### preparar la tabla final


#### a) tabla place

In [58]:
final_df_colnames_place = ["Place of interest","Type of place","Place address","Place distrit","Place neighborhood","Place description","Place transport","Place latitude","Place longitude"]
len(final_df_colnames_place)

9

In [59]:
final_df_place = parques_municipales_df[["NOMBRE","Type of Place","NOMBRE-VIA","DISTRITO", "BARRIO", "DESCRIPCION-ENTIDAD", "TRANSPORTE","LATITUD","LONGITUD"]]
final_df_place.columns = final_df_colnames_place


#### mercator place

In [60]:
def to_mercator2(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [61]:
final_df_place = final_df_place
final_df_place["mercator place"] = final_df_place.apply(lambda final_df_place: to_mercator2(final_df_place["Place latitude"], final_df_place["Place longitude"]), axis=1)

In [62]:
%%time
final_df_place

Wall time: 0 ns


,Place of interest,Type of place,Place address,Place distrit,Place neighborhood,Place description,Place transport,Place latitude,Place longitude,mercator place
0,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839)
1,Jardines San Francisco el Grande,Principales parques y jardines municipales,GRAN VIA DE SAN FRANCISCO,CENTRO,PALACIO,Este jardín se encuentra en el solar de 4.200 ...,"Bus: 3, 60 y 148. Servicio Bicimad, para cons...",40.407143,-3.712167,POINT (4498102.554322098 -413525.9071853098)
2,Jardines de Andrés Saborit,Principales parques y jardines municipales,RAIMUNDO FERNANDEZ VILLAVERDE,CHAMBERI,RIOS ROSAS,Jardines situados en la calle Raimundo Fernánd...,"Metro: Nuevos Ministerios (líneas 6, 8 y 10) B...",40.446133,-3.698970,POINT (4502442.959942039 -412053.7810488821)
3,Jardines de El Buen Retiro,Principales parques y jardines municipales,INDEPENDENCIA,RETIRO,LOS JERONIMOS,Jardín Histórico Artístico creado en el siglo ...,"Metro: Retiro (línea 2), Príncipe de Vergara (...",40.419641,-3.687934,POINT (4499493.840154874 -410822.6776250893)
4,Jardines de Gloria Fuertes,Principales parques y jardines municipales,MADRE DE DIOS,CHAMARTIN,NUEVA ESPAÑA,Jardín dedicado a la poetisa y escritora de cu...,"Metro: línea 9 (Colombia) Bus: 16,29,150 Serv...",40.461518,-3.675118,POINT (4504155.540151379 -409393.078164072)
...,...,...,...,...,...,...,...,...,...,...
198,Vivero Estufas de El Retiro,Principales parques y jardines municipales,ANGEL CAIDO,RETIRO,LOS JERONIMOS,Recinto construido en 1889 para albergar diver...,"Metro: Atocha (línea 1), Ibiza (línea 9) Bus: ...",40.409899,-3.682331,POINT (4498409.408289636 -410197.6442728437)
199,Vivero de la Casa de Campo,Principales parques y jardines municipales,AZUL,MONCLOA-ARAVACA,CASA DE CAMPO,La primera noticia documental la tenemos en 18...,"Metro: Príncipe Pío (líneas 6 y 10 y ramal), L...",40.419842,-3.728697,POINT (4499516.183398882 -415369.9370911405)
200,Vivero municipal de Migas Calientes,Principales parques y jardines municipales,M-30,MONCLOA-ARAVACA,CIUDAD UNIVERSITARIA,Centro de producción donde se realiza el culti...,NaN,40.445356,-3.739974,POINT (4502356.383233474 -416627.9974384935)
201,Zona verde C/ Fresnedillas,Principales parques y jardines municipales,FRESNEDILLAS,FUENCARRAL-EL PARDO,FUENTELARREINA,Esta nueva zona verde está situada entre las c...,NaN,40.477264,-3.743003,POINT (4505908.402877585 -416965.8212977283)


#### b) tabla bicimad

In [63]:
final_df_colnames_bici = ["BiciMAD station","Station location","Station latitude","Station longitude","Station base availability","Station Bikes Availability"]

len(final_df_colnames_bici)
#FUTURE COLUMN = "Distance Between BiciMAD station and Place of interest"

6

In [64]:
final_df_bici = bicimad_st_df[["name","address","LATITUD","LONGITUD", "no_available", "dock_bikes"]]
final_df_bici.columns = final_df_colnames_bici
final_df_bici

,BiciMAD station,Station location,Station latitude,Station longitude,Station base availability,Station Bikes Availability
0,Puerta del Sol A,Puerta del Sol nº 1,40.4172137,-3.7018341,0,15
1,Puerta del Sol B,Puerta del Sol nº 1,40.41731271011562,-3.701602938060457,0,22
2,Miguel Moya,Calle Miguel Moya nº 1,40.4205886,-3.7058415,0,12
3,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.4302937,-3.7069171,0,4
4,Malasaña,Calle Manuela Malasaña nº 5,40.4285524,-3.7025875,0,12
...,...,...,...,...,...,...
259,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.44375,-3.72699,0,0
260,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.44342,-3.72693,0,12
261,Facultad Biología,Calle José Antonio Novais frente al nº 12,40.4483322,-3.7272945,0,9
262,Facultad Derecho,Avenida Complutense nº 23,40.45109,-3.72937,0,5


#### change availability

In [65]:
def change_availability(data):
    if data == 0:
        return "disponible"
    elif data == 1:
        return "no disponible"
    else:
        return data

In [66]:
final_df_bici["Station base availability"] = final_df_bici["Station base availability"].apply(change_availability)

#### mercator station


In [67]:
%%time
final_df_bici = final_df_bici
final_df_bici["Station latitude"] = pd.to_numeric(final_df_bici["Station latitude"])
final_df_bici["Station longitude"] = pd.to_numeric(final_df_bici["Station longitude"])
final_df_bici["mercator station"] = final_df_bici.apply(lambda final_df_bici: to_mercator2(final_df_bici["Station latitude"], final_df_bici["Station longitude"]), axis=1)

Wall time: 16.7 s


In [68]:
final_df_bici

,BiciMAD station,Station location,Station latitude,Station longitude,Station base availability,Station Bikes Availability,mercator station
0,Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,disponible,15,POINT (4499223.648366921 -412373.2851572837)
1,Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,disponible,22,POINT (4499234.670122574 -412347.498528782)
2,Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,disponible,12,POINT (4499599.340516399 -412820.3206106317)
3,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,disponible,4,POINT (4500679.707306497 -412940.3068148255)
4,Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,disponible,12,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...,...,...,...
259,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990,disponible,0,POINT (4502177.655770458 -415179.5220676512)
260,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930,disponible,12,POINT (4502140.920338497 -415172.8287427764)
261,Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295,disponible,9,POINT (4502687.743941171 -415213.4906984274)
262,Facultad Derecho,Avenida Complutense nº 23,40.451090,-3.729370,disponible,5,POINT (4502994.74083288 -415445.0243226733)


#### c) tabla unificada completa con todas las posibilidades de coordenadas

In [69]:
%%time
#another way to
final_df = final_df_place.assign(key=1).merge(final_df_bici.assign(key=1), how='outer', on = 'key')
final_df

Wall time: 48.9 ms


,Place of interest,Type of place,Place address,Place distrit,Place neighborhood,Place description,Place transport,Place latitude,Place longitude,mercator place,key,BiciMAD station,Station location,Station latitude,Station longitude,Station base availability,Station Bikes Availability,mercator station
0,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,disponible,15,POINT (4499223.648366921 -412373.2851572837)
1,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,disponible,22,POINT (4499234.670122574 -412347.498528782)
2,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,disponible,12,POINT (4499599.340516399 -412820.3206106317)
3,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,disponible,4,POINT (4500679.707306497 -412940.3068148255)
4,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,disponible,12,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53587,Área Forestal de Tres Cantos,Principales parques y jardines municipales,M-607,FUENCARRAL-EL PARDO,EL PARDO,El Monte de Tres Cantos se sitúa al este del t...,NaN,40.582363,-3.705431,POINT (4517607.930659908 -412774.5800167918),1,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990,disponible,0,POINT (4502177.655770458 -415179.5220676512)
53588,Área Forestal de Tres Cantos,Principales parques y jardines municipales,M-607,FUENCARRAL-EL PARDO,EL PARDO,El Monte de Tres Cantos se sitúa al este del t...,NaN,40.582363,-3.705431,POINT (4517607.930659908 -412774.5800167918),1,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930,disponible,12,POINT (4502140.920338497 -415172.8287427764)
53589,Área Forestal de Tres Cantos,Principales parques y jardines municipales,M-607,FUENCARRAL-EL PARDO,EL PARDO,El Monte de Tres Cantos se sitúa al este del t...,NaN,40.582363,-3.705431,POINT (4517607.930659908 -412774.5800167918),1,Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295,disponible,9,POINT (4502687.743941171 -415213.4906984274)
53590,Área Forestal de Tres Cantos,Principales parques y jardines municipales,M-607,FUENCARRAL-EL PARDO,EL PARDO,El Monte de Tres Cantos se sitúa al este del t...,NaN,40.582363,-3.705431,POINT (4517607.930659908 -412774.5800167918),1,Facultad Derecho,Avenida Complutense nº 23,40.451090,-3.729370,disponible,5,POINT (4502994.74083288 -415445.0243226733)


In [70]:
#classic way with for in
"""
final_df_colnames_total = ["Place of interest","Type of place","Place address","Place distrit","Place neighborhood","Place description","Place transport","Place latitude","Place longitude","mercator place","BiciMAD station","Station location","Station latitude","Station longitude","Station base availability","Station Bikes Availability","mercator station"]

final_df_list =[]
for i in range(len(final_df_place)): #203 filas
        for e in range(len(final_df_bici)): # 264 filas
            #raw_number = i*e #53592 filas
            #final_df.loc[raw_number] = final_df_place.loc[i].append(final_df_bici.loc[e])
            str_individual = final_df_place.loc[i].append(final_df_bici.loc[e])
            list_individual = str_individual.tolist()
            final_df_list.append(list_individual)
            
final_df = pd.DataFrame(final_df_list,columns=final_df_colnames_total)
final_df
"""

'\nfinal_df_colnames_total = ["Place of interest","Type of place","Place address","Place distrit","Place neighborhood","Place description","Place transport","Place latitude","Place longitude","mercator place","BiciMAD station","Station location","Station latitude","Station longitude","Station base availability","Station Bikes Availability","mercator station"]\n\nfinal_df_list =[]\nfor i in range(len(final_df_place)): #203 filas\n        for e in range(len(final_df_bici)): # 264 filas\n            #raw_number = i*e #53592 filas\n            #final_df.loc[raw_number] = final_df_place.loc[i].append(final_df_bici.loc[e])\n            str_individual = final_df_place.loc[i].append(final_df_bici.loc[e])\n            list_individual = str_individual.tolist()\n            final_df_list.append(list_individual)\n            \nfinal_df = pd.DataFrame(final_df_list,columns=final_df_colnames_total)\nfinal_df\n'

In [71]:
def distance_meters(start,finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    #start = to_mercator2(lat_start, long_start)
    #finish = to_mercator2(lat_finish, long_finish)
    return start.distance(finish)

In [72]:
%%time
final_df["Distance Between BiciMAD station and Place of interest"] = final_df.apply(lambda final_df: distance_meters(final_df["mercator place"], final_df["mercator station"]), axis=1)

Wall time: 960 ms


In [73]:
final_df

,Place of interest,Type of place,Place address,Place distrit,Place neighborhood,Place description,Place transport,Place latitude,Place longitude,mercator place,key,BiciMAD station,Station location,Station latitude,Station longitude,Station base availability,Station Bikes Availability,mercator station,Distance Between BiciMAD station and Place of interest
0,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,disponible,15,POINT (4499223.648366921 -412373.2851572837),3160.930400
1,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,disponible,22,POINT (4499234.670122574 -412347.498528782),3133.481106
2,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,disponible,12,POINT (4499599.340516399 -412820.3206106317),3360.222755
3,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,disponible,4,POINT (4500679.707306497 -412940.3068148255),3178.932155
4,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,disponible,12,POINT (4500485.866677178 -412457.3286497012),2731.560930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53587,Área Forestal de Tres Cantos,Principales parques y jardines municipales,M-607,FUENCARRAL-EL PARDO,EL PARDO,El Monte de Tres Cantos se sitúa al este del t...,NaN,40.582363,-3.705431,POINT (4517607.930659908 -412774.5800167918),1,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990,disponible,0,POINT (4502177.655770458 -415179.5220676512),15616.565866
53588,Área Forestal de Tres Cantos,Principales parques y jardines municipales,M-607,FUENCARRAL-EL PARDO,EL PARDO,El Monte de Tres Cantos se sitúa al este del t...,NaN,40.582363,-3.705431,POINT (4517607.930659908 -412774.5800167918),1,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930,disponible,12,POINT (4502140.920338497 -415172.8287427764),15651.837120
53589,Área Forestal de Tres Cantos,Principales parques y jardines municipales,M-607,FUENCARRAL-EL PARDO,EL PARDO,El Monte de Tres Cantos se sitúa al este del t...,NaN,40.582363,-3.705431,POINT (4517607.930659908 -412774.5800167918),1,Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295,disponible,9,POINT (4502687.743941171 -415213.4906984274),15118.209452
53590,Área Forestal de Tres Cantos,Principales parques y jardines municipales,M-607,FUENCARRAL-EL PARDO,EL PARDO,El Monte de Tres Cantos se sitúa al este del t...,NaN,40.582363,-3.705431,POINT (4517607.930659908 -412774.5800167918),1,Facultad Derecho,Avenida Complutense nº 23,40.451090,-3.729370,disponible,5,POINT (4502994.74083288 -415445.

In [74]:
final_df_full_info_csv_optimizated = final_df.to_csv("datasets/final_df_full_info_optimizated.csv", sep=',')

In [75]:
total_distance_array_min = final_df.groupby(["Place of interest"])["Distance Between BiciMAD station and Place of interest"].min().tolist()

In [76]:
"""
total_distance = final_df["Distance Between BiciMAD station and Place of interest"].tolist()

total_distance_array = np.array(total_distance).reshape(len(final_df_bici),len(final_df_place))

total_distance_array_min = []
for i in total_distance_array:
    total_distance_array_min.append(i.min())
total_distance_array_min
"""

'\ntotal_distance = final_df["Distance Between BiciMAD station and Place of interest"].tolist()\n\ntotal_distance_array = np.array(total_distance).reshape(len(final_df_bici),len(final_df_place))\n\ntotal_distance_array_min = []\nfor i in total_distance_array:\n    total_distance_array_min.append(i.min())\ntotal_distance_array_min\n'

In [77]:
list_min=[]
for i in total_distance_array_min:
    linea_para_df = final_df.loc[final_df["Distance Between BiciMAD station and Place of interest"]==i]
    linea_para_df_list = linea_para_df.values.tolist()[0]
    list_min.append(linea_para_df_list)

In [78]:
final_df_colnames_total = ["Place of interest","Type of place","Place address","Place distrit","Place neighborhood","Place description","Place transport","Place latitude","Place longitude","mercator place","key","BiciMAD station","Station location","Station latitude","Station longitude","Station base availability","Station Bikes Availability","mercator station"]

final_df_min_distance = pd.DataFrame(list_min, columns=final_df_colnames_total+["Distance Between BiciMAD station and Place of interest"])
final_df_min_distance

,Place of interest,Type of place,Place address,Place distrit,Place neighborhood,Place description,Place transport,Place latitude,Place longitude,mercator place,key,BiciMAD station,Station location,Station latitude,Station longitude,Station base availability,Station Bikes Availability,mercator station,Distance Between BiciMAD station and Place of interest
0,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),1,Diego de León 52,Calle Diego de León nº 52,40.434597,-3.678492,disponible,13,POINT (4501158.781867075 -409769.4558894613),126.921708
1,Jardines San Francisco el Grande,Principales parques y jardines municipales,GRAN VIA DE SAN FRANCISCO,CENTRO,PALACIO,Este jardín se encuentra en el solar de 4.200 ...,"Bus: 3, 60 y 148. Servicio Bicimad, para cons...",40.407143,-3.712167,POINT (4498102.554322098 -413525.9071853098),1,Puerta de Toledo,Glorieta de la Puerta de Toledo nº 1,40.407036,-3.711051,disponible,8,POINT (4498090.649721576 -413401.4898074972),124.985613
2,Jardines de Andrés Saborit,Principales parques y jardines municipales,RAIMUNDO FERNANDEZ VILLAVERDE,CHAMBERI,RIOS ROSAS,Jardines situados en la calle Raimundo Fernánd...,"Metro: Nuevos Ministerios (líneas 6, 8 y 10) B...",40.446133,-3.698970,POINT (4502442.959942039 -412053.7810488821),1,Raimundo Fernández,Calle Raimundo Fernández Villaverde nº 33,40.447125,-3.700167,disponible,9,POINT (4502553.359051885 -412187.3054398999),173.253359
3,Jardines de El Buen Retiro,Principales parques y jardines municipales,INDEPENDENCIA,RETIRO,LOS JERONIMOS,Jardín Histórico Artístico creado en el siglo ...,"Metro: Retiro (línea 2), Príncipe de Vergara (...",40.419641,-3.687934,POINT (4499493.840154874 -410822.6776250893),1,Plaza de la Independencia,Plaza de la Independencia nº 6,40.419752,-3.688398,disponible,19,POINT (4499506.210630401 -410874.469478817),53.248707
4,Jardines de Gloria Fuertes,Principales parques y jardines municipales,MADRE DE DIOS,CHAMARTIN,NUEVA ESPAÑA,Jardín dedicado a la poetisa y escritora de cu...,"Metro: línea 9 (Colombia) Bus: 16,29,150 Serv...",40.461518,-3.675118,POINT (4504155.540151379 -409393.078164072),1,Pío XII,Avenida Pío XII nº 3,40.460410,-3.677120,disponible,11,POINT (4504032.238487074 -409616.4103570179),255.108935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Vivero Estufas de El Retiro,Principales parques y jardines municipales,ANGEL CAIDO,RETIRO,LOS JERONIMOS,Recinto construido en 1889 para albergar diver...,"Metro: Atocha (línea 1), Ibiza (línea 9) Bus: ...",40.409899,-3.682331,POINT (4498409.408289636 -410197.6442728437),1,Menéndez Pelayo,Avenida de Menéndez Pelayo con calle Poeta Est...,40.408280,-3.678484,disponible,3,POINT (4498229.209091766 -409768.5411851413),465.404352
199,Vivero de la Casa de Campo,Principales parques y jardines municipales,AZUL,MONCLOA-ARAVACA,CASA DE CAMPO,La primera noticia documental la tenemos en 18...,"Metro: Príncipe Pío (líneas 6 y 10 y ramal), L...",40.419842,-3.728697,POINT (4499516.183398882 -415369.9370911405),1,Puerta del Ángel,Plaza Puerta del Ángel nº 4,40.413764,-3.728318,disponible,16,POINT (4498839.629519532 -415327.6677751078),677.873031
200,Vivero municipal de Migas Calientes,Principales parques y jardines municipales,M-30,MONCLOA-ARAVACA,CIUDAD UNIVERSITARIA,Centro de producción donde se realiza el culti...,NaN,40.445356,-3.739974,POINT (4502356.383233474 -416627.9974384935),1,INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970,disponible,14,POINT (4501644.435409558 -415511.9577777162),1323.787834
201,Zona verde C/ Fresnedillas,Principales parques y jardines municipales,FRESNEDILLAS,FUENCARRAL-EL PARDO,FUENTELARREINA,Esta nueva zona verde está situada entre las c...,NaN,40.477264,-3.743003,POINT (4505908.402877585 -416965.8

In [79]:
#comprobacion
min(final_df.iloc[0:264,:]["Distance Between BiciMAD station and Place of interest"])

126.92170781350367

In [80]:
final_df_min_distance_csv_optimizated = final_df_min_distance.to_csv("datasets/final_df_min_distance_optimizated.csv", sep=',')

## extra

In [113]:
#https://www.google.com/maps/dir/Prta+del+Sol/Vivero+Municipal+de+la+Casa+de+Campo

In [110]:
def place_maps_func(data):
    place_maps = "+".join(data.split(" "))
    return place_maps

In [111]:
def g_maps_func(place_maps, bici_maps):
    g_maps = "https://www.google.com/maps/dir/" + str(place_maps) + "/" + str(bici_maps)
    return g_maps

In [112]:
final_df_min_distance["place_maps"] = final_df_min_distance["Place of interest"].apply(place_maps_func)
final_df_min_distance["bici_maps"] = final_df_min_distance["Station location"].apply(place_maps_func)
final_df_min_distance["g_maps"] = final_df_min_distance.apply(lambda final_df_min_distance: g_maps_func(final_df_min_distance["place_maps"], final_df_min_distance["bici_maps"]), axis=1)

final_df_min_distance

,Place of interest,Type of place,Place address,Place distrit,Place neighborhood,Place description,Place transport,Place latitude,Place longitude,mercator place,...,Station location,Station latitude,Station longitude,Station base availability,Station Bikes Availability,mercator station,Distance Between BiciMAD station and Place of interest,place_maps,bici_maps,g_maps
0,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),...,Calle Diego de León nº 52,40.434597,-3.678492,disponible,13,POINT (4501158.781867075 -409769.4558894613),126.921708,Jardines+Gregorio+Ordóñez,Calle+Diego+de+León+nº+52,https://www.google.com/maps/dir/Jardines+Grego...
1,Jardines San Francisco el Grande,Principales parques y jardines municipales,GRAN VIA DE SAN FRANCISCO,CENTRO,PALACIO,Este jardín se encuentra en el solar de 4.200 ...,"Bus: 3, 60 y 148. Servicio Bicimad, para cons...",40.407143,-3.712167,POINT (4498102.554322098 -413525.9071853098),...,Glorieta de la Puerta de Toledo nº 1,40.407036,-3.711051,disponible,8,POINT (4498090.649721576 -413401.4898074972),124.985613,Jardines+San+Francisco+el+Grande,Glorieta+de+la+Puerta+de+Toledo+nº+1,https://www.google.com/maps/dir/Jardines+San+F...
2,Jardines de Andrés Saborit,Principales parques y jardines municipales,RAIMUNDO FERNANDEZ VILLAVERDE,CHAMBERI,RIOS ROSAS,Jardines situados en la calle Raimundo Fernánd...,"Metro: Nuevos Ministerios (líneas 6, 8 y 10) B...",40.446133,-3.698970,POINT (4502442.959942039 -412053.7810488821),...,Calle Raimundo Fernández Villaverde nº 33,40.447125,-3.700167,disponible,9,POINT (4502553.359051885 -412187.3054398999),173.253359,Jardines+de+Andrés+Saborit,Calle+Raimundo+Fernández+Villaverde+nº+33,https://www.google.com/maps/dir/Jardines+de+An...
3,Jardines de El Buen Retiro,Principales parques y jardines municipales,INDEPENDENCIA,RETIRO,LOS JERONIMOS,Jardín Histórico Artístico creado en el siglo ...,"Metro: Retiro (línea 2), Príncipe de Vergara (...",40.419641,-3.687934,POINT (4499493.840154874 -410822.6776250893),...,Plaza de la Independencia nº 6,40.419752,-3.688398,disponible,19,POINT (4499506.210630401 -410874.469478817),53.248707,Jardines+de+El+Buen+Retiro,Plaza+de+la+Independencia+nº+6,https://www.google.com/maps/dir/Jardines+de+El...
4,Jardines de Gloria Fuertes,Principales parques y jardines municipales,MADRE DE DIOS,CHAMARTIN,NUEVA ESPAÑA,Jardín dedicado a la poetisa y escritora de cu...,"Metro: línea 9 (Colombia) Bus: 16,29,150 Serv...",40.461518,-3.675118,POINT (4504155.540151379 -409393.078164072),...,Avenida Pío XII nº 3,40.460410,-3.677120,disponible,11,POINT (4504032.238487074 -409616.4103570179),255.108935,Jardines+de+Gloria+Fuertes,Avenida+Pío+XII+nº+3,https://www.google.com/maps/dir/Jardines+de+Gl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Vivero Estufas de El Retiro,Principales parques y jardines municipales,ANGEL CAIDO,RETIRO,LOS JERONIMOS,Recinto construido en 1889 para albergar diver...,"Metro: Atocha (línea 1), Ibiza (línea 9) Bus: ...",40.409899,-3.682331,POINT (4498409.408289636 -410197.6442728437),...,Avenida de Menéndez Pelayo con calle Poeta Est...,40.408280,-3.678484,disponible,3,POINT (4498229.209091766 -409768.5411851413),465.404352,Vivero+Estufas+de+El+Retiro,Avenida+de+Menéndez+Pelayo+con+calle+Poeta+Est...,https://www.google.com/maps/dir/Vivero+Estufas...
199,Vivero de la Casa de Campo,Principales parques y jardines municipales,AZUL,MONCLOA-ARAVACA,CASA DE CAMPO,La primera noticia documental la tenemos en 18...,"Metro: Príncipe Pío (líneas 6 y 10 y ramal), L...",40.419842,-3.728697,POINT (4499516.183398882 -415369.9370911405),...,Plaza Puerta del Ángel nº 4,40.413764,-3.728318,disponible,16,POINT (4498839.629519532 -415327.6677751078),677.873031,Vivero+de+la+Casa+de+Campo,Plaza+

### open street maps

In [139]:
open_street_df = pd.DataFrame(final_df_min_distance["mercator place"])
open_street_df

,mercator place
0,POINT (4501032.386132487 -409780.9987743839)
1,POINT (4498102.554322098 -413525.9071853098)
2,POINT (4502442.959942039 -412053.7810488821)
3,POINT (4499493.840154874 -410822.6776250893)
4,POINT (4504155.540151379 -409393.078164072)
...,...
198,POINT (4498409.408289636 -410197.6442728437)
199,POINT (4499516.183398882 -415369.9370911405)
200,POINT (4502356.383233474 -416627.9974384935)
201,POINT (4505908.402877585 -416965.8212977283)


In [152]:
def lat_long_open_street0(geo):
    open_geo = str(geo).replace("POINT (","").replace(")","").split(" ")
    open_point_0 = open_geo[0].strip()
    return open_point_0

In [153]:
def lat_long_open_street1(geo):
    open_geo = str(geo).replace("POINT (","").replace(")","").split(" ")
    open_point_1 = open_geo[1].strip()
    return open_point_1

In [160]:
open_street_df["open_0"] = open_street_df["mercator place"].apply(lat_long_open_street0)
open_street_df["open_1"] = open_street_df["mercator place"].apply(lat_long_open_street1)
open_street_df["Place latitude"] = final_df_min_distance["Place latitude"]
open_street_df["Place longitude"] = final_df_min_distance["Place longitude"]

open_street_df


,mercator place,open_0,open_1,Place latitude,Place longitude
0,POINT (4501032.386132487 -409780.9987743839),4501032.386132487,-409780.9987743839,40.433462,-3.678595
1,POINT (4498102.554322098 -413525.9071853098),4498102.554322098,-413525.9071853098,40.407143,-3.712167
2,POINT (4502442.959942039 -412053.7810488821),4502442.959942039,-412053.7810488821,40.446133,-3.698970
3,POINT (4499493.840154874 -410822.6776250893),4499493.840154874,-410822.6776250893,40.419641,-3.687934
4,POINT (4504155.540151379 -409393.078164072),4504155.540151379,-409393.078164072,40.461518,-3.675118
...,...,...,...,...,...
198,POINT (4498409.408289636 -410197.6442728437),4498409.408289636,-410197.6442728437,40.409899,-3.682331
199,POINT (4499516.183398882 -415369.9370911405),4499516.183398882,-415369.9370911405,40.419842,-3.728697
200,POINT (4502356.383233474 -416627.9974384935),4502356.383233474,-416627.9974384935,40.445356,-3.739974
201,POINT (4505908.402877585 -416965.8212977283),4505908.402877585,-416965.8212977283,40.477264,-3.743003


In [192]:
m = folium.Map([40.433462, -3.678595], zoom_start=12)
locs = zip(open_street_df["Place latitude"].tolist(), open_street_df["Place longitude"].tolist())
for location in locs:
    #print(location)
    if "nan" not in str(location):
        folium.CircleMarker(location=location).add_to(m)
        m.save('open_street_df.html')

In [193]:
m

In [252]:
def fw_ratio(x):
    fw1 = x
    fw2 = user_imput
    ratio = fuzz.ratio(user_imput.lower().strip(), x.lower().strip())
    return ratio

In [253]:
user_imput = " vichwero estufas de El   Reero"

In [254]:
final_df_min_distance["fw_ratio"] = final_df_min_distance["Place of interest"].apply(fw_ratio)

In [255]:
final_df_min_distance

,Place of interest,Type of place,Place address,Place distrit,Place neighborhood,Place description,Place transport,Place latitude,Place longitude,mercator place,...,Station latitude,Station longitude,Station base availability,Station Bikes Availability,mercator station,Distance Between BiciMAD station and Place of interest,place_maps,bici_maps,g_maps,fw_ratio
0,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),...,40.434597,-3.678492,disponible,13,POINT (4501158.781867075 -409769.4558894613),1.269217e+02,Jardines+Gregorio+Ordóñez,Calle+Diego+de+León+nº+52,https://www.google.com/maps/dir/Jardines+Grego...,22
1,Jardines San Francisco el Grande,Principales parques y jardines municipales,GRAN VIA DE SAN FRANCISCO,CENTRO,PALACIO,Este jardín se encuentra en el solar de 4.200 ...,"Bus: 3, 60 y 148. Servicio Bicimad, para cons...",40.407143,-3.712167,POINT (4498102.554322098 -413525.9071853098),...,40.407036,-3.711051,disponible,8,POINT (4498090.649721576 -413401.4898074972),1.249856e+02,Jardines+San+Francisco+el+Grande,Glorieta+de+la+Puerta+de+Toledo+nº+1,https://www.google.com/maps/dir/Jardines+San+F...,39
2,Jardines de Andrés Saborit,Principales parques y jardines municipales,RAIMUNDO FERNANDEZ VILLAVERDE,CHAMBERI,RIOS ROSAS,Jardines situados en la calle Raimundo Fernánd...,"Metro: Nuevos Ministerios (líneas 6, 8 y 10) B...",40.446133,-3.698970,POINT (4502442.959942039 -412053.7810488821),...,40.447125,-3.700167,disponible,9,POINT (4502553.359051885 -412187.3054398999),1.732534e+02,Jardines+de+Andrés+Saborit,Calle+Raimundo+Fernández+Villaverde+nº+33,https://www.google.com/maps/dir/Jardines+de+An...,32
3,Jardines de El Buen Retiro,Principales parques y jardines municipales,INDEPENDENCIA,RETIRO,LOS JERONIMOS,Jardín Histórico Artístico creado en el siglo ...,"Metro: Retiro (línea 2), Príncipe de Vergara (...",40.419641,-3.687934,POINT (4499493.840154874 -410822.6776250893),...,40.419752,-3.688398,disponible,19,POINT (4499506.210630401 -410874.469478817),5.324871e+01,Jardines+de+El+Buen+Retiro,Plaza+de+la+Independencia+nº+6,https://www.google.com/maps/dir/Jardines+de+El...,54
4,Jardines de Gloria Fuertes,Principales parques y jardines municipales,MADRE DE DIOS,CHAMARTIN,NUEVA ESPAÑA,Jardín dedicado a la poetisa y escritora de cu...,"Metro: línea 9 (Colombia) Bus: 16,29,150 Serv...",40.461518,-3.675118,POINT (4504155.540151379 -409393.078164072),...,40.460410,-3.677120,disponible,11,POINT (4504032.238487074 -409616.4103570179),2.551089e+02,Jardines+de+Gloria+Fuertes,Avenida+Pío+XII+nº+3,https://www.google.com/maps/dir/Jardines+de+Gl...,39
5,Jardines de José Luis Jiménez Manso,Principales parques y jardines municipales,CAMINO DE LOS VINATEROS,MORATALAZ,MEDIA LEGUA,Parque emblemático de la zona al ser uno de lo...,"Metro: Estrella (línea 9) Bus: 20, 30, 32 Ser...",40.410945,-3.660502,POINT (4498525.813556252 -407762.6743641385),...,40.410530,-3.657790,disponible,3,POINT (4498479.622286306 -407460.1888912669),3.059920e+02,Jardines+de+José+Luis+Jiménez+Manso,Calle+Camino+de+los+Vinateros+nº+28,https://www.google.com/maps/dir/Jardines+de+Jo...,31
6,Jardines de Perón,Principales parques y jardines municipales,GENERAL PERON,TETUAN,CUATRO CAMINOS,Jardines situados en la avenida General Perón ...,"Metro: Santiago Bernabéu (línea 10) Bus: 3, 5,...",40.452576,-3.695782,POINT (4503160.152133536 -411698.1348104622),...,40.454846,-3.694622,disponible,11,POINT (4503412.812312503 -411568.7402199105),2.838664e+02,Jardines+de+Perón,Calle+Orense+nº+36,https://www.google.com/maps/dir/Jardines+de+Pe...,38
7,Jardines de Sabatini,Principales parques y jardines municipales,BAILEN,CENTRO,PALACIO,Los Jardines de Sabatini se encuentran situado...,"Metro: Opera Bus: 2, 146 Servicio Bicimad,...",40.419911,-3.713185,POINT (4499523.94238939 -413639.5

In [256]:
pd.set_option('display.max_rows', 500)
final_df_min_distance["fw_ratio"].max()

84

In [257]:
#que nos devuelva en función de el maximo el nombre del parque que debería ser y su bici